In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler

train_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_train.csv', header=None)
test_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_test.csv', header=None)

X_train = train_df.iloc[:, :187].values
y_train = train_df.iloc[:, 187].values
X_test = test_df.iloc[:, :187].values
y_test = test_df.iloc[:, 187].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 1024

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
class SimpleModel1D(nn.Module):
    def __init__(self, num_classes):
        super(SimpleModel1D, self).__init__()

        self.features = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )

        self.adaptive_pool = nn.AdaptiveAvgPool1d(1)

        self.classifier = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.adaptive_pool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Load the CSV files
train_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_train.csv', header=None)
test_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_test.csv', header=None)

# Map of original class labels
classes = {0: 'N', 1: 'S', 2: 'V', 3: 'F', 4: 'Q'}

# Add a new column for 'normal' vs 'not normal'
train_df['binary_label'] = train_df[187].apply(lambda x: 0 if x == 0 else 1)
test_df['binary_label'] = test_df[187].apply(lambda x: 0 if x == 0 else 1)

# Prepare the data
X_train = train_df.iloc[:, :187].values
y_train = train_df.iloc[:, 187].values
y_train_binary = train_df['binary_label'].values

X_test = test_df.iloc[:, :187].values
y_test = test_df.iloc[:, 187].values
y_test_binary = test_df['binary_label'].values

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_train_binary = torch.tensor(y_train_binary, dtype=torch.long)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)
y_test_binary = torch.tensor(y_test_binary, dtype=torch.long)

batch_size = 1024

train_dataset = TensorDataset(X_train, y_train_binary, y_train)
test_dataset = TensorDataset(X_test, y_test_binary, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Hyperparameters
lr = 0.001
num_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleModel1D(num_classes=5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for i, (inputs, binary_labels, original_labels) in enumerate(train_loader):
            inputs, original_labels = inputs.unsqueeze(1).to(device), original_labels.to(device)  # Add a channel dimension

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, original_labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss/(i+1))
            pbar.update(1)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

def evaluate_model(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, binary_labels, original_labels in data_loader:
            inputs = inputs.unsqueeze(1).to(device)
            original_labels = original_labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, original_labels)
            total_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == original_labels).sum().item()
            total_samples += inputs.size(0)

    average_loss = total_loss / total_samples
    accuracy = total_correct / total_samples

    return average_loss, accuracy

test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Epoch 1/10:   0%|          | 0/86 [00:00<?, ?batch/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
Epoch 1/10: 100%|██████████| 86/86 [00:02<00:00, 34.32batch/s, loss=0.811]


Epoch 1/10, Loss: 0.8105609063492265


Epoch 2/10: 100%|██████████| 86/86 [00:02<00:00, 36.34batch/s, loss=0.611]


Epoch 2/10, Loss: 0.6111201831074649


Epoch 3/10: 100%|██████████| 86/86 [00:02<00:00, 39.26batch/s, loss=0.537]


...

Epoch 8/10, Loss: 0.3003607090822486


Epoch 9/10: 100%|██████████| 86/86 [00:03<00:00, 27.75batch/s, loss=0.282]


Epoch 9/10, Loss: 0.2821058667329855


Epoch 10/10: 100%|██████████| 86/86 [00:02<00:00, 29.25batch/s, loss=0.266]


Epoch 10/10, Loss: 0.2662139368611713
Test Loss: 0.2599, Test Accuracy: 0.9318


In [ ]:
import os
import sys
import torch
from torch.utils.data import TensorDataset, DataLoader
import subprocess
import numpy as np
import pandas as pd
import glob
from collections import OrderedDict
import random

# Clone and import CNNWordReco repository
if not os.path.isdir('CNNWordReco'):
    subprocess.call(['git', 'clone', 'https://github.com/saztorralba/CNNWordReco'])
if 'CNNWordReco' not in sys.path:
    sys.path.append('CNNWordReco')
from utils.cnn_func import load_data, train_model, validate_model, test_model
from test_wordreco import show_matrix

# Arguments
args = {
    'cv_percentage': 0.1,
    'xsize': 20,
    'ysize': 20,
    'num_blocks': 10,
    'channels': 32,
    'dropout': 0.3,
    'embedding_size': 128,
    'epochs': 20,
    'batch_size': 32,
    'learning_rate': 0.001,
    'seed': 0,
    'device': 'cpu',
    'verbose': 1,
    'augment': False,
    'vocab': OrderedDict({'ZERO': 0, 'ONE': 1, 'TWO': 2, 'THREE': 3, 'FOUR': 4, 'FIVE': 5, 'SIX': 6, 'SEVEN': 7, 'EIGHT': 8, 'NINE': 9})
}

# Initialise the random seeds
random.seed(args['seed'])
torch.manual_seed(args['seed'])
torch.cuda.manual_seed(args['seed'])
torch.backends.cudnn.deterministic = True

# Read data and store in dataframe
wavfiles = glob.glob('./datasets/recordings/*.wav')
speakers = [file.split('/')[-1].split('_')[1] for file in wavfiles]
words = [list(args['vocab'].keys())[int(file.split('/')[-1].split('_')[0])] for file in wavfiles]
rec_number = [int(file.split('/')[-1].split('_')[2].split('.')[0]) for file in wavfiles]
data = pd.DataFrame({'wavfile': wavfiles, 'speaker': speakers, 'word': words, 'rec_number': rec_number})

print('Training model with recordings [5-49] from all speakers')
# Load data
train_data = data.loc[data['rec_number'] >= 5].reset_index(drop=True)
test_data = data.loc[data['rec_number'] < 5].reset_index(drop=True)
trainset, validset, trainlabels, validlabels = load_data(train_data, True, **args)
args['mean'] = torch.mean(trainset.float())
args['std'] = torch.std(trainset.float())

X_train = torch.tensor(trainset, dtype=torch.float32)
y_train = torch.tensor(trainlabels, dtype=torch.long)

X_train = X_train.view(X_train.size(0), 1, -1)
train_dataset = TensorDataset(X_train, y_train)
batch_size = 512  # Choose the batch size you want to use

train_loader_voice = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

sample_batch = next(iter(train_loader_voice))
print("Shape of input batch:", sample_batch[0].shape)
print("Shape of target batch:", sample_batch[1].shape)


Training model with recordings [5-49] from all speakers
Shape of input batch: torch.Size([512, 1, 400])
Shape of target batch: torch.Size([512])


In [ ]:
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fonction pour extraire les caractéristiques
def extract_features(model, data_loader):
    features = []
    labels_list = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)

            outputs = model(inputs)
            features.append(outputs)
            labels_list.append(labels)

    features = torch.cat(features)
    labels_list = torch.cat(labels_list)
    return features, labels_list

# Extract features and labels from test data
test_features, test_labels = extract_features(model, train_loader_voice)

# Calculate cosine similarity
distances = cosine_similarity(test_features.detach().cpu(), test_features.detach().cpu())
np.fill_diagonal(distances, float('-inf'))

# Fonction pour trouver les indices des éléments les plus similaires
def find_max_indices(arr, k):
    if k > len(arr):
        raise ValueError("k cannot be greater than the size of the array")
    idx = np.argpartition(arr, -k)[-k:]
    sorted_idx = np.argsort(arr[idx])[::-1]
    return idx[sorted_idx]

# Fonction pour calculer la précision top-k
def compute_top_k_accuracy(distances, test_labels, k):
    correct = 0
    for id_, elem in enumerate(distances):
        indices = find_max_indices(elem, k)
        candidates = [test_labels[idx].item() for idx in indices]
        if test_labels[id_].item() in candidates:
            correct += 1
    accuracy = correct / len(distances)
    return accuracy

# Compute top-k accuracy
real_top_accuracies = []
for k in range(1, 6):
    accuracy = compute_top_k_accuracy(distances, test_labels, k)
    real_top_accuracies.append(accuracy)
    print(f"Top-{k} : {accuracy:.4f}")


Top-1 : 0.2432
Top-2 : 0.3835
Top-3 : 0.4700
Top-4 : 0.5510
Top-5 : 0.6058
